In [1]:
### 라이브러리 import 
import matplotlib.pyplot as plt
import librosa.display
import librosa
import numpy as np
import glob
import os
import multiprocessing
import pandas as pd

import seaborn as sns
from os.path import join
from tensorflow import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, Conv1D, MaxPooling1D, Bidirectional, GRU, Dense, Dropout, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 불러오기

In [2]:
import os
import pandas as pd

# 파일을 읽어올 디렉토리 경로를 설정합니다.
base_folder_path = 'D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/원천데이터'

# 모든 파일의 경로와 라벨을 저장할 리스트를 초기화합니다.
data = []

# 디렉토리와 하위 디렉토리를 순회하며 모든 파일의 경로를 찾습니다.
for dirpath, dirnames, filenames in os.walk(base_folder_path):
     for filename in filenames:
        # 파일의 전체 경로를 생성합니다.
        file_path = os.path.join(dirpath, filename)
        # 상위 폴더명을 라벨로 지정합니다.
        label = os.path.basename(os.path.dirname(os.path.dirname(file_path)))[2:]
        # 파일의 경로와 라벨을 리스트에 추가합니다.
        data.append([file_path, label])

# 리스트를 데이터 프레임으로 변환합니다.
df = pd.DataFrame(data, columns=['file_path', 'emotion'])


In [3]:
### zip 파일 경로 제거
df = df[2:]

In [4]:
df.reset_index(inplace=True,drop=True)
df

,file_path,emotion
0,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨
1,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨
2,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨
3,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨
4,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨
...,...,...
453960,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립
453961,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립
453962,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립
453963,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립


In [5]:
df_train = df

In [6]:
df_train['emotion'].value_counts()

기쁨    66097
불안    65853
분노    65715
슬픔    64359
당황    64201
중립    64000
상처    63740
Name: emotion, dtype: int64

In [7]:
label_name = df_train['emotion'].unique()
label_name

array(['기쁨', '슬픔', '분노', '불안', '상처', '당황', '중립'], dtype=object)

In [8]:
label_dic = {}
for i in range(len(label_name)):
    label_dic[label_name[i]]=i
    
label_dic

{'기쁨': 0, '슬픔': 1, '분노': 2, '불안': 3, '상처': 4, '당황': 5, '중립': 6}

In [9]:
label_list = []
for i in range(len(df_train)):
    label_list.append(label_dic[df_train.iloc[i]['emotion']])


In [10]:
df_train['label']=0
df_train['label']=label_list

In [11]:
df_train

,file_path,emotion,label
0,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨,0
1,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨,0
2,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨,0
3,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨,0
4,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,기쁨,0
...,...,...,...
453960,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립,6
453961,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립,6
453962,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립,6
453963,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,중립,6


In [12]:
### 메모리 부족으로
# 각 감정당 20000 개씩

df_train_기쁨 = df_train[df_train['emotion']=='기쁨'].reset_index(drop=True)
df_train_슬픔 = df_train[df_train['emotion']=='슬픔'].reset_index(drop=True)
df_train_분노 = df_train[df_train['emotion']=='분노'].reset_index(drop=True)
df_train_불안 = df_train[df_train['emotion']=='불안'].reset_index(drop=True)
df_train_상처 = df_train[df_train['emotion']=='상처'].reset_index(drop=True)
df_train_당황 = df_train[df_train['emotion']=='당황'].reset_index(drop=True)
df_train_중립 = df_train[df_train['emotion']=='중립'].reset_index(drop=True)



emolist = [df_train_기쁨,df_train_슬픔,df_train_분노,df_train_불안,df_train_상처,df_train_당황,df_train_중립]

In [13]:
df_train_당황

,file_path,emotion,label
0,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
1,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
2,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
3,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
4,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
...,...,...,...
64196,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
64197,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
64198,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5
64199,D:/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training...,당황,5


# 데이터 전처리

In [14]:
# 데이터 로드 확인
y, sr = librosa.load(df_train['file_path'][10])
y, sr

(array([-4.8561702e-05, -7.8174460e-05, -1.3746883e-04, ...,
         6.5956017e-05,  9.5696181e-05,  0.0000000e+00], dtype=float32),
 22050)

In [14]:
### 데이터 로드
# 메모리 부족 sr 제한  sr=48000
def load(audio_file):
    y, sr = librosa.load(audio_file)
    return y

In [16]:
### 데이터 로드 사용
# 20000 개 데이터 사용
audio_files = []
for df_emo in emolist:
    for j in range(1000):
        audio_file = load(df_emo['file_path'][j])
        audio_files.append(audio_file)

In [17]:
audio_files

[array([ 0.00036851,  0.00050837,  0.00031012, ..., -0.00023282,
        -0.00028146,  0.        ], dtype=float32),
 array([1.4124501e-04, 2.1723934e-04, 1.8026942e-04, ..., 1.3207902e-04,
        7.7469624e-05, 0.0000000e+00], dtype=float32),
 array([-3.6089306e-05, -7.6349570e-05, -8.1596198e-05, ...,
         2.7347593e-05,  1.0819806e-04,  0.0000000e+00], dtype=float32),
 array([ 1.4007412e-04,  1.9398678e-04,  2.0063081e-04, ...,
        -4.8765331e-05, -9.2775350e-05,  0.0000000e+00], dtype=float32),
 array([-2.8035105e-05, -8.3711260e-05, -1.8048549e-05, ...,
        -1.8532816e-04, -1.1324887e-04,  0.0000000e+00], dtype=float32),
 array([-3.1149415e-05,  1.8725834e-05,  1.8918949e-05, ...,
        -1.4510519e-04, -1.2563299e-04, -1.7442182e-04], dtype=float32),
 array([1.1310979e-04, 1.2320127e-04, 7.1443814e-05, ..., 1.0330454e-04,
        6.5338638e-05, 6.3269523e-05], dtype=float32),
 array([-4.1085073e-05, -5.7392899e-05, -3.8901926e-05, ...,
        -4.5022674e-04, -4.5091

In [14]:
### 정규화
def nor(audio_np):
    normed_wav = audio_np / max(np.abs(audio_np))
    return normed_wav

In [19]:
### 정규화 사용
audio_nor_data = []
for i in range(len(audio_files)):
    audio_nor = nor(audio_files[i])
    audio_nor_data.append(audio_nor)

In [15]:
def extract_mfccs(audio_file):
    
    y_pre = librosa.effects.preemphasis(audio_file, coef=0.97)
    
    n_fft = 1024
    hop_length = 256
    win_length = 512
    window = 'hamming'
    n_mels = 128
    n_mfcc = 64

    mfccs = librosa.feature.mfcc(y=y_pre,win_length = win_length , sr=22050, n_mfcc=n_mfcc, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, window=window)
    return mfccs

In [21]:
audio_mfcc = []
for i in range(len(audio_nor_data)):
    mfcc = extract_mfccs(audio_nor_data[i])
    audio_mfcc.append(mfcc)

In [22]:
audio_mfcc[0].shape

(64, 351)

In [23]:
# 평균 길이 구하기
mean_len = 0
for i in range(len(audio_mfcc)):
    mean_len += len(audio_mfcc[i][1])
    
mean_len = mean_len/len(audio_mfcc)
print(mean_len)

291.1972857142857


In [28]:
### 메모리 정리
audio_files = 0
audio_nor_data = 0
import gc
gc.collect()

In [43]:
### 패딩
# dtype='float32' 안하면 int32 가 기본 그럼 소수점이 짤려 데이터 정확도 낮아짐
train_seq = []
for i in range(len(audio_mfcc)):
    audio_seq = pad_sequences(audio_mfcc[i],padding='post' , truncating='post',maxlen=300, dtype='float32')
    train_seq.append(audio_seq)

In [45]:
train_ar=np.array(train_seq)
train_ar.shape
train_input = train_ar

In [46]:
### 갯수조절해서 타겟값 넣기
train_target = []
for df_emo in emolist:
    for j in range(1000):
        train_target.append(df_emo['label'][j])
        
train_target = np.array(train_target)

In [48]:
train_input.shape, train_target.shape

((7000, 64, 300), (7000,))

In [49]:
### 데이터 스플릿
# stratify=y : 클래스 비율 유지

train_input, val_input, train_target, val_target =\
train_test_split(train_input, train_target, test_size=0.2, random_state=42,
                stratify=train_target)

In [50]:
train_input.shape, train_target.shape

((5600, 64, 300), (5600,))

# 모델 학습

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, ReLU, Softmax

model = Sequential()

# 1. conv block
model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(64, 300, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 2. conv block
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 3. conv block
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 4. conv block
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# flatten the output of the conv block
model.add(Flatten())

# output layer
model.add(Dense(7, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:

checkpoint_cb = keras.callbacks.ModelCheckpoint("C:/pknu_6/model1_weight/model_{epoch}.h5", save_best_only=False)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                 restore_best_weights=True)

In [18]:
history = model.fit(train_input, train_target, epochs=200, batch_size=16,
                   validation_data=(val_input, val_target),
                   callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/200
350/350 [==============================] - 18s 16ms/step - loss: 1.7506 - accuracy: 0.5991 - val_loss: 3.0745 - val_accuracy: 0.5207
Epoch 2/200
350/350 [==============================] - 5s 14ms/step - loss: 0.7260 - accuracy: 0.8239 - val_loss: 1.7460 - val_accuracy: 0.6886
Epoch 3/200
350/350 [==============================] - 5s 14ms/step - loss: 0.5874 - accuracy: 0.8666 - val_loss: 1.5078 - val_accuracy: 0.7650
Epoch 4/200
350/350 [==============================] - 5s 13ms/step - loss: 0.3564 - accuracy: 0.9116 - val_loss: 0.6767 - val_accuracy: 0.8764
Epoch 5/200
350/350 [==============================] - 5s 14ms/step - loss: 0.2735 - accuracy: 0.9350 - val_loss: 1.1561 - val_accuracy: 0.8529
Epoch 6/200
350/350 [==============================] - 5s 14ms/step - loss: 0.2717 - accuracy: 0.9423 - val_loss: 1.0046 - val_accuracy: 0.8636
Epoch 7/200
350/350 [==============================] - 5s 14ms/step - loss: 0.2336 - accuracy: 0.9454 - val_loss: 0.9309 - val_accuracy

In [ ]:
# 커널 restart 후 가중치 불러오고 학습 이어서하기

In [14]:
### 데이터 로드
# 메모리 부족 sr 제한  sr=48000
def load(audio_file):
    y, sr = librosa.load(audio_file)
    return y
### 정규화
def nor(audio_np):
    normed_wav = audio_np / max(np.abs(audio_np))
    return normed_wav

def extract_mfccs(audio_file):
    
    y_pre = librosa.effects.preemphasis(audio_file, coef=0.97)
    
    n_fft = 1024
    hop_length = 256
    win_length = 512
    window = 'hamming'
    n_mels = 128
    n_mfcc = 64

    mfccs = librosa.feature.mfcc(y=y_pre,win_length = win_length , sr=22050, n_mfcc=n_mfcc, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, window=window)
    return mfccs

In [15]:
### 데이터 로드 사용
# 20000 개 데이터 사용
audio_files = []
for df_emo in emolist:
    for j in range(45000,46000):
        audio_file = load(df_emo['file_path'][j])
        audio_files.append(audio_file)

### 정규화 사용
audio_nor_data = []
for i in range(len(audio_files)):
    audio_nor = nor(audio_files[i])
    audio_nor_data.append(audio_nor)

audio_mfcc = []
for i in range(len(audio_nor_data)):
    mfcc = extract_mfccs(audio_nor_data[i])
    audio_mfcc.append(mfcc)

### 패딩
# dtype='float32' 안하면 int32 가 기본 그럼 소수점이 짤려 데이터 정확도 낮아짐
train_seq = []
for i in range(len(audio_mfcc)):
    audio_seq = pad_sequences(audio_mfcc[i],padding='post' , truncating='post',maxlen=300, dtype='float32')
    train_seq.append(audio_seq)

train_ar=np.array(train_seq)
train_ar.shape
train_input = train_ar

### 갯수조절해서 타겟값 넣기
train_target = []
for df_emo in emolist:
    for j in range(45000,46000):
        train_target.append(df_emo['label'][j])
        
train_target = np.array(train_target)

print(train_input.shape, train_target.shape)

### 데이터 스플릿
# stratify=y : 클래스 비율 유지

train_input, val_input, train_target, val_target =\
train_test_split(train_input, train_target, test_size=0.2, random_state=42,
                stratify=train_target)

print(train_input.shape, train_target.shape)

(7000, 64, 300) (7000,)
(5600, 64, 300) (5600,)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, ReLU, Softmax

model = Sequential()

# 1. conv block
model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(64, 300, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 2. conv block
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 3. conv block
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 4. conv block
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# flatten the output of the conv block
model.add(Flatten())

# output layer
model.add(Dense(7, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [16]:
model.load_weights("C:/pknu_6/model4_weight/model_9.h5")

In [17]:

checkpoint_cb = keras.callbacks.ModelCheckpoint("C:/pknu_6/model5_weight/model_{epoch}.h5", save_best_only=False)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                 restore_best_weights=True)

In [19]:
history = model.fit(train_input, train_target, epochs=200, batch_size=16,
                   validation_data=(val_input, val_target),
                   callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/200
350/350 [==============================] - 19s 18ms/step - loss: 0.5337 - accuracy: 0.9089 - val_loss: 0.1069 - val_accuracy: 0.9700
Epoch 2/200
350/350 [==============================] - 5s 14ms/step - loss: 0.1596 - accuracy: 0.9580 - val_loss: 0.0545 - val_accuracy: 0.9821
Epoch 3/200
350/350 [==============================] - 5s 14ms/step - loss: 0.0964 - accuracy: 0.9704 - val_loss: 0.0392 - val_accuracy: 0.9864
Epoch 4/200
350/350 [==============================] - 5s 14ms/step - loss: 0.0765 - accuracy: 0.9787 - val_loss: 0.0234 - val_accuracy: 0.9921
Epoch 5/200
350/350 [==============================] - 5s 14ms/step - loss: 0.0510 - accuracy: 0.9836 - val_loss: 0.0194 - val_accuracy: 0.9929
Epoch 6/200
350/350 [==============================] - 5s 14ms/step - loss: 0.0419 - accuracy: 0.9855 - val_loss: 0.0169 - val_accuracy: 0.9936
Epoch 7/200
350/350 [==============================] - 5s 14ms/step - loss: 0.0418 - accuracy: 0.9864 - val_loss: 0.0147 - val_accuracy

In [16]:
### 데이터 로드
# 메모리 부족 sr 제한  sr=48000
def load(audio_file):
    y, sr = librosa.load(audio_file)
    return y
### 정규화
def nor(audio_np):
    normed_wav = audio_np / max(np.abs(audio_np))
    return normed_wav

def extract_mfccs(audio_file):
    
    y_pre = librosa.effects.preemphasis(audio_file, coef=0.97)
    
    n_fft = 1024
    hop_length = 256
    win_length = 512
    window = 'hamming'
    n_mels = 128
    n_mfcc = 64

    mfccs = librosa.feature.mfcc(y=y_pre,win_length = win_length , sr=22050, n_mfcc=n_mfcc, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, window=window)
    return mfccs

In [17]:
def preprocess_audio_file(file_path):
    audio_file = load(file_path)
    audio_nor = nor(audio_file)
    mfcc = extract_mfccs(audio_nor)
    audio_seq = pad_sequences(mfcc, padding='post' , truncating='post', maxlen=300, dtype='float32')
    return audio_seq


In [18]:
def create_dataset(file_paths, labels):
    def generator():
        for file_path, label in zip(file_paths, labels):
            yield preprocess_audio_file(file_path), label

    dataset = tf.data.Dataset.from_generator(generator, 
                                             output_signature=(tf.TensorSpec(shape=(64, 300), dtype=tf.float32),
                                                              tf.TensorSpec(shape=(), dtype=tf.int32)))
    return dataset


In [21]:
# 파일 경로와 레이블을 리스트로 불러옵니다.
file_paths = [df_emo['file_path'][j] for df_emo in emolist for j in range(0,63000)]
labels = [df_emo['label'][j] for df_emo in emolist for j in range(0,63000)]

# 훈련 데이터와 테스트 데이터를 분할합니다.
train_file_paths, val_file_paths, train_labels, val_labels =\
train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# 데이터 생성기를 생성합니다.
train_dataset = create_dataset(train_file_paths, train_labels)
val_dataset = create_dataset(val_file_paths, val_labels)

# 모델을 훈련시킵니다.
history = model.fit(train_dataset.batch(16), epochs=200,
                    validation_data=val_dataset.batch(16),
                    callbacks=[checkpoint_cb, early_stopping_cb])


Epoch 1/200
22050/22050 [==============================] - 9074s 411ms/step - loss: 0.6014 - accuracy: 0.7796 - val_loss: 0.4059 - val_accuracy: 0.8521
Epoch 2/200
22050/22050 [==============================] - 8759s 397ms/step - loss: 0.6017 - accuracy: 0.7803 - val_loss: 0.4085 - val_accuracy: 0.8510
Epoch 3/200
22050/22050 [==============================] - 8971s 407ms/step - loss: 0.5985 - accuracy: 0.7807 - val_loss: 0.4072 - val_accuracy: 0.8523
Epoch 4/200
22050/22050 [==============================] - 9186s 417ms/step - loss: 0.5968 - accuracy: 0.7814 - val_loss: 0.4029 - val_accuracy: 0.8537
Epoch 5/200
22050/22050 [==============================] - 9217s 418ms/step - loss: 0.5969 - accuracy: 0.7814 - val_loss: 0.4065 - val_accuracy: 0.8528
Epoch 6/200
22050/22050 [==============================] - 9229s 419ms/step - loss: 0.5945 - accuracy: 0.7827 - val_loss: 0.4091 - val_accuracy: 0.8506
Epoch 7/200
13838/22050 [=================>............] - ETA: 46:55 - loss: 0.5950 - a

In [19]:
model.load_weights('C:/pknu_6/model5_weight/model_15.h5')


# 파일 경로와 레이블을 리스트로 불러옵니다.
file_paths = [df_emo['file_path'][j] for df_emo in emolist for j in range(0,63000)]
labels = [df_emo['label'][j] for df_emo in emolist for j in range(0,63000)]

# 훈련 데이터와 테스트 데이터를 분할합니다.
train_file_paths, val_file_paths, train_labels, val_labels =\
train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# 데이터 생성기를 생성합니다.
train_dataset = create_dataset(train_file_paths, train_labels)
val_dataset = create_dataset(val_file_paths, val_labels)

# 훈련 세트에서의 손실과 정확도를 계산합니다.
train_results = model.evaluate(train_dataset.batch(16))
print(f"훈련 손실: {train_results[0]}")
print(f"훈련 정확도: {train_results[1]}")  # 백분율이 아닌 원래 값으로 출력

# 검증 세트에서의 손실과 정확도를 계산합니다.
val_results = model.evaluate(val_dataset.batch(16))
print(f"검증 손실: {val_results[0]}")
print(f"검증 정확도: {val_results[1]}")  # 백분율이 아닌 원래 값으로 출력

22050/22050 [==============================] - 6489s 294ms/step - loss: 0.3513 - accuracy: 0.8743
훈련 손실: 0.3512919545173645
훈련 정확도: 0.8742517232894897
5513/5513 [==============================] - 1618s 293ms/step - loss: 0.3983 - accuracy: 0.8560
검증 손실: 0.3983461856842041
검증 정확도: 0.8560090661048889


In [26]:
import matplotlib.pyplot as plt
#6 훈련 과정 시각화 (정확도)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 7 훈련 과정 시각화 (손실)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [27]:
plt.show()

In [29]:
print(history.history)